In [1]:
import pickle
import glob
import numpy as np
from coffea import processor
from coffea import util
from coffea.lookup_tools.dense_lookup import dense_lookup

In [2]:
# folder with b_tag_efficiency_processor output files
output_histograms = "/home/cms-jovyan/wprime_plus_b/outfiles/2023-04-12/btag_eff/2017/mu"

# load btag efficiency processor outputs 
outputs = []
for file in glob.glob(f"{output_histograms}/*.pkl"):
    with open(file, "rb") as f:
        outputs.append(pickle.load(f))
        
outputs[0]

{'WJetsToLNu_HT-200To400': {'sumw': 42400348.0,
  'eff_hist': Hist(
    Regular(20, 20, 500, name='pt'),
    Regular(4, 0, 2.5, name='abseta'),
    IntCategory([0, 4, 5], name='flavor'),
    Regular(2, 0, 2, name='passWP'),
    storage=Weight()) # Sum: WeightedSum(value=1.5212e+08, variance=1.5212e+08) (WeightedSum(value=1.52121e+08, variance=1.52121e+08) with flow)}}

In [3]:
LUMI = 41477.877399
XSECS = {
    "TTTo2L2Nu": 88.29,
    "TTToHadronic": 377.96,
    "TTToSemiLeptonic": 365.34,
    "ST_s-channel_4f_leptonDecays": 3.549,
    "ST_t-channel_antitop_4f_InclusiveDecays": 69.09,
    "ST_t-channel_antitop_5f_InclusiveDecays": 71.74,
    "ST_t-channel_top_4f_InclusiveDecays": 115.3,
    "ST_t-channel_top_5f_InclusiveDecays": 119.7,
    "ST_tW_antitop_5f_inclusiveDecays": 34.97,
    "ST_tW_top_5f_inclusiveDecays": 34.91,
    "WJetsToLNu_HT-100To200": 1395,
    "WJetsToLNu_HT-200To400": 407.9,
    "WJetsToLNu_HT-400To600": 57.48,
    "WJetsToLNu_HT-600To800": 12.87,
    "WJetsToLNu_HT-800To1200": 5.366,
    "WJetsToLNu_HT-1200To2500": 1.074,
    "WJetsToLNu_HT-2500ToInf": 0.008001,
    "DYJetsToLL_M-50_HT-100to200": 160.7,
    "DYJetsToLL_M-50_HT-200to400": 48.63,
    "DYJetsToLL_M-50_HT-400to600": 6.993,
    "DYJetsToLL_M-50_HT-600to800": 1.761,
    "DYJetsToLL_M-50_HT-800to1200": 0.8021,
    "DYJetsToLL_M-50_HT-1200to2500": 0.1937,
    "DYJetsToLL_M-50_HT-2500toInf": 0.003514,
    "WW": 75.95,
    "WZ": 27.59,
    "ZZ": 12.17 
}

effs = []
for out in outputs:
    for dataset, values in out.items():
        # select eff histogram
        eff_hist = values["eff_hist"] 
        
        # since weighted histograms can't be divided as eff_hist[{"passWP": True}] / eff_hist[{"passWP": sum}]
        # we compute the efficiency using the underlying numpy arrays
        eff = eff_hist[{'passWP':sum}].copy() # Making a copy of the histogram without the 'passWP' axis
        num = eff_hist[{"passWP": True}].view(flow=True)['value']
        den = eff_hist[{"passWP": sum}].view(flow=True)['value']
        eff.view(flow=True)['value'] = np.divide(num, den, out=np.ones_like(num * den) * 1.0, where=den != 0) # to avoid nan values
        
        # apply the appropriate cross section/effective lumi weightings
        weight = (XSECS[dataset] * LUMI) / values["sumw"]
        effs.append(eff * weight)
        
# accumulate histograms 
effs = processor.accumulate(effs)
effs

Hist(
  Regular(20, 20, 500, name='pt'),
  Regular(4, 0, 2.5, name='abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  storage=Weight()) # Sum: WeightedSum(value=266.872, variance=2.48409e+08) (WeightedSum(value=1197.23, variance=2.48439e+08) with flow)

The `flow=True` arguments ensures that efficiencies are also calculated for the overflow and underflow bins if they are needed.

In [4]:
# create lookup table from the efficiency histogram
efflookup = dense_lookup(effs.values(), [ax.edges for ax in effs.axes])  
efflookup

3 dimensional histogram with axes:
	1: [ 20.  44.  68.  92. 116. 140. 164. 188. 212. 236. 260. 284. 308. 332.
 356. 380. 404. 428. 452. 476. 500.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3.]

In [5]:
# Efficiency at 43 GeV, |eta|=0.2, for light, c, and b quark respectively
efflookup(43, 0.1, np.array([0, 1, 2]))

array([0.0894443 , 0.60543691, 2.43813612])

Notice that hadron flavours (0, 4, 5) turn into (0, 1, 2). 

In [6]:
efflookup._axes[-1] = np.array([0., 4., 5., 6.]) # change (0, 1, 2) to (0, 4, 5)
efflookup

3 dimensional histogram with axes:
	1: [ 20.  44.  68.  92. 116. 140. 164. 188. 212. 236. 260. 284. 308. 332.
 356. 380. 404. 428. 452. 476. 500.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 4. 5. 6.]

In [7]:
# Efficiency at 43 GeV, |eta|=0.2, for light, c, and b quark respectively
efflookup(43, 0.1, np.array([0, 4, 5]))

array([0.0894443 , 0.60543691, 2.43813612])

In [8]:
# save lookup table
output_path = f"/home/cms-jovyan/wprime_plus_b/wprime_plus_b/data/btag_eff_deepJet_M_2017.coffea"
util.save(efflookup, output_path)